### 📊 Análisis Exploratorio Global – Casos Judiciales
# Ministerios Públicos – Argentina

## Introduccion
En este notebook se realiza un análisis exploratorio global de los casos judiciales iniciados por los Ministerios Públicos en Argentina.

El objetivo es obtener una visión panorámica del volumen, la distribución temporal y geográfica, y la composición penal de los casos, así como una primera aproximación a la completitud del registro de información.

Este análisis es descriptivo y exploratorio; no se realizan inferencias causales ni evaluaciones normativas.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 5)

# Rutas (notebook/ -> ../data)
DATA_DIR = Path("..") / "data" / "eda"
DATA_FILE = DATA_DIR / "casos_eda.csv"

df = pd.read_csv(DATA_FILE, low_memory=False)

print("Dataset cargado")
df.shape

: 

## Descripción general del dataset

Se presenta una descripción básica del dataset, incluyendo dimensiones, período temporal cubierto y nivel de agregación.

In [ ]:
print(f"Filas: {df.shape[0]:,}")
print(f"Columnas: {df.shape[1]}")

df[["caso_fecha_inicio"]].describe(datetime_is_numeric=True)
df.head()


## Volumen total de casos

Se analiza la magnitud del fenómeno judicial a partir del número total de casos únicos registrados.


In [ ]:
total_casos = df["caso_id"].nunique()
total_registros = len(df)

print(f"Casos únicos: {total_casos:,}")
print(f"Registros totales: {total_registros:,}")

# Nota: Un mismo `caso_id` puede aparecer en múltiples registros debido a la existencia de más de un delito asociado a un mismo caso.


## Distribución de casos por año

Se analiza la evolución temporal de los casos judiciales iniciados, utilizando la fecha de inicio del caso.


In [ ]:
df["anio_inicio"] = pd.to_datetime(df["caso_fecha_inicio"], errors="coerce").dt.year

casos_por_anio = (
    df.groupby("anio_inicio")["caso_id"]
    .nunique()
    .reset_index(name="casos")
    .dropna()
)
casos_por_anio

sns.lineplot(data=casos_por_anio, x="anio_inicio", y="casos", marker="o")
plt.title("Casos judiciales iniciados por año")
plt.xlabel("Año")
plt.ylabel("Cantidad de casos")
plt.show()


## Distribución geográfica por provincia

Se explora la distribución de los casos judiciales iniciados según la provincia.


In [ ]:
casos_por_provincia = (
    df.groupby("provincia_nombre")["caso_id"]
    .nunique()
    .sort_values(ascending=False)
)
casos_por_provincia.head(10)

top_provincias = casos_por_provincia.head(10).reset_index()

sns.barplot(
    data=top_provincias,
    x="caso_id",
    y="provincia_nombre",
    orient="h"
)
plt.title("Top 10 provincias por cantidad de casos")
plt.xlabel("Cantidad de casos")
plt.ylabel("Provincia")
plt.show()


# Advertencia: Estos valores representan volumen de causas iniciadas, no tasas de criminalidad.


## Composición penal de los casos

Se analiza la distribución de los delitos más frecuentes registrados en el dataset.


In [ ]:
top_delitos = (
    df[df["delito_descripcion"] != "No informado"]
    .groupby("delito_descripcion")["caso_id"]
    .nunique()
    .sort_values(ascending=False)
    .head(15)
)
top_delitos

top_delitos_df = top_delitos.reset_index()

sns.barplot(
    data=top_delitos_df,
    x="caso_id",
    y="delito_descripcion",
    orient="h"
)
plt.title("Top 15 delitos más frecuentes")
plt.xlabel("Cantidad de casos")
plt.ylabel("Delito")
plt.show()

## Completitud del registro de información

Se evalúa la proporción de registros con información disponible en variables clave.


In [ ]:
variables_clave = [
    "caso_fecha_hecho",
    "caso_hora_hecho",
    "delito_codigo",
    "unidad_descripcion"
]

completitud = (
    df[variables_clave]
    .notna()
    .mean()
    .sort_values()
    * 100
)

completitud

completitud.plot(kind="barh")
plt.title("Completitud de variables clave (%)")
plt.xlabel("Porcentaje de registros con dato disponible")
plt.ylabel("Variable")
plt.show()


# Nota: La ausencia de información en estas variables no implica error, sino que refleja la lógica y las etapas del proceso judicial.


## Hallazgos preliminares

• El volumen de casos judiciales iniciados es elevado y presenta variaciones interanuales claras.  
• La distribución geográfica muestra una fuerte concentración en un grupo reducido de provincias.  
• La composición penal está dominada por un conjunto limitado de delitos frecuentes.  
• La completitud del registro varía significativamente según la variable analizada, siendo la hora del hecho una de las más incompletas.  
• Los valores faltantes observados reflejan características estructurales del sistema de registro judicial.
